In [20]:
from pyspark.ml.feature import Word2Vec, Word2VecModel
from pyspark.sql import functions as F
import numpy as np
from pyspark.ml.linalg import Vectors

In [2]:
model = Word2VecModel.load('/common/users/shared/cs543_fall22_group3/models/word2vec')
vector_table = model.getVectors()

/koko/system/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [13]:
processed_df = spark.read.json('/common/users/shared/cs543_fall22_group3/combined/combined_processed')

In [14]:
processed_df.drop('selected_text').coalesce(1).write.format('text').option('header', 'false').mode('overwrite').save('/common/users/shared/cs543_fall22_group3/combined/tokens.txt')

In [31]:
vectors_df = None

for line in open('/common/users/shared/cs543_fall22_group3/combined/tokens.txt/part-00000-9c2229f7-e3bb-4f88-9baa-31962493e3d6-c000.txt'):
    
    tokens = line.strip().split(',')
    vector_list = vector_table.filter(vector_table.word.isin(tokens)).drop('word').collect()
    if vectors_df is None:
        vectors_df = spark.createDataFrame([(Vectors.dense([v[0][0] for v in vector_list]), 1.0)], ["features", "weighCol"])
    else:
        temp = spark.createDataFrame([(Vectors.dense([v[0][0] for v in vector_list]), 1.0)], ["features", "weighCol"])
        vectors_df = vectors_df.union(temp)

In [35]:
vectors_df.write.mode("Overwrite").json('/common/users/shared/cs543_fall22_group3/combined/combined_vectors')